# 6-4. 실습 (1) 데이터 다듬기

In [1]:
import os, re 
import numpy as np
import tensorflow as tf
import pandas as pd

# 파일을 읽기모드로 열고
# 라인 단위로 끊어서 list 형태로 읽어옵니다.
from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/aiffel/lyricist/data/shakespeare.txt'
with open(file_path, "r") as f:
    raw_corpus = f.read().splitlines()

# 앞에서부터 10라인만 화면에 출력해 볼까요?
print(raw_corpus[:9])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['First Citizen:', 'Before we proceed any further, hear me speak.', '', 'All:', 'Speak, speak.', '', 'First Citizen:', 'You are all resolved rather to die than to famish?', '']


In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

Before we proceed any further, hear me speak.
Speak, speak.
You are all resolved rather to die than to famish?


In [3]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [4]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> before we proceed any further , hear me speak . <end>',
 '<start> speak , speak . <end>',
 '<start> you are all resolved rather to die than to famish ? <end>',
 '<start> resolved . resolved . <end>',
 '<start> first , you know caius marcius is chief enemy to the people . <end>',
 '<start> we know t , we know t . <end>',
 '<start> let us kill him , and we ll have corn at our own price . <end>',
 '<start> is t a verdict ? <end>',
 '<start> no more talking on t let it be done away , away ! <end>',
 '<start> one word , good citizens . <end>']

In [5]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  143   40 ...    0    0    0]
 [   2  110    4 ...    0    0    0]
 [   2   11   50 ...    0    0    0]
 ...
 [   2  149 4553 ...    0    0    0]
 [   2   34   71 ...    0    0    0]
 [   2  945   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f83fc0ea950>


In [6]:
print(tensor[:3, :10])

[[   2  143   40  933  140  591    4  124   24  110]
 [   2  110    4  110    5    3    0    0    0    0]
 [   2   11   50   43 1201  316    9  201   74    9]]


In [7]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : .
6 : the
7 : and
8 : i
9 : to
10 : of


In [8]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2 143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0
   0   0]
[143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0   0
   0   0]


In [9]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset element_spec=(TensorSpec(shape=(256, 20), dtype=tf.int32, name=None), TensorSpec(shape=(256, 20), dtype=tf.int32, name=None))>

# 6-5. 실습 (2) 인공지능 학습시키기

In [10]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [11]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 20, 7001), dtype=float32, numpy=
array([[[-1.9697910e-04,  2.5575518e-04,  3.5115832e-04, ...,
          1.8613836e-04,  1.1259153e-04, -3.4533219e-05],
        [-2.1533310e-04,  2.8350501e-04,  6.9371768e-04, ...,
          3.3618600e-04,  4.9604551e-04, -1.6401125e-05],
        [-2.6111875e-04,  3.8149286e-04,  9.0915384e-04, ...,
          4.4032582e-04,  2.8339005e-04,  2.6473310e-04],
        ...,
        [-2.7635214e-03,  6.9706803e-03,  2.9073004e-03, ...,
          1.5362573e-03, -2.0373582e-04,  1.6753789e-04],
        [-2.7654383e-03,  7.5432253e-03,  3.0638871e-03, ...,
          1.6098974e-03, -7.6594668e-05,  2.2722603e-04],
        [-2.7457541e-03,  8.0586467e-03,  3.2077259e-03, ...,
          1.6730599e-03,  3.3304910e-05,  2.9389068e-04]],

       [[-1.9697910e-04,  2.5575518e-04,  3.5115832e-04, ...,
          1.8613836e-04,  1.1259153e-04, -3.4533219e-05],
        [-3.9050827e-04,  2.8913206e-04,  4.6891932e-04, ...,
          2.9404738e-04, -

In [12]:
model.summary()

Model: "text_generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  1792256   
                                                                 
 lstm (LSTM)                 multiple                  5246976   
                                                                 
 lstm_1 (LSTM)               multiple                  8392704   
                                                                 
 dense (Dense)               multiple                  7176025   
                                                                 
Total params: 22,607,961
Trainable params: 22,607,961
Non-trainable params: 0
_________________________________________________________________


In [13]:
# optimizer와 loss등은 차차 배웁니다 
# 혹시 미리 알고 싶다면 아래 문서를 참고하세요

# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)

# model.fit() 함수에 들어가는 다양한 인자를 알고 싶다면 아래의 문서를 참고하세요. 
# https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit

model.fit(dataset, epochs=30)

Epoch 1/30
93/93 [==============================] - 12s 103ms/step - loss: 3.4610
Epoch 2/30
93/93 [==============================] - 10s 103ms/step - loss: 2.7937
Epoch 3/30
93/93 [==============================] - 10s 103ms/step - loss: 2.6816
Epoch 4/30
93/93 [==============================] - 10s 103ms/step - loss: 2.5875
Epoch 5/30
93/93 [==============================] - 10s 103ms/step - loss: 2.5281
Epoch 6/30
70/93 [=====================>........] - ETA: 2s - loss: 2.4699

KeyboardInterrupt: ignored

# 6-7. 프로젝트: 멋진 작사가 만들기

In [14]:
import glob
import os, re 
import numpy as np
import tensorflow as tf
import pandas as pd

# 파일을 읽기모드로 열고
# 라인 단위로 끊어서 list 형태로 읽어옵니다.
from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/aiffel/lyricist/data/lyrics/*'
txt_list = glob.glob(file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])  

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
데이터 크기: 187088
Examples:
 ["Let's stay together I, I'm I'm so in love with you", 'Whatever you want to do', 'Is all right with me']


In [16]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.

    if idx > 9: break   # 문장 10개 확인
        
    print(sentence)

Let's stay together I, I'm I'm so in love with you
Whatever you want to do
Is all right with me
Cause you make me feel so brand new
And I want to spend my life with you Let me say that since, baby, since we've been together
Loving you forever
Is what I need
Let me, be the one you come running to
I'll never be untrue Oh baby
Let's, let's stay together (gether)


In [17]:
#토큰화
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 소문자로 바꾸고, 양쪽 공백을 지웁니다
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 특수문자 양쪽에 공백을 넣고
    sentence = re.sub(r'[" "]+', " ", sentence) # 여러개의 공백은 하나의 공백으로 바꿉니다
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
    sentence = sentence.strip() # 다시 양쪽 공백을 지웁니다
    sentence = '<start> ' + sentence + ' <end>' # 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
    return sentence

In [18]:
#정제 데이터 구축하기
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue #길이 0
    if len(sentence.split()) >= 13: continue  #15개 이하(start,end포함)
    
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    
    
corpus[:10] #정제결과 확인

['<start> let s stay together i , i m i m so in love with you <end>',
 '<start> whatever you want to do <end>',
 '<start> is all right with me <end>',
 '<start> cause you make me feel so brand new <end>',
 '<start> loving you forever <end>',
 '<start> is what i need <end>',
 '<start> let me , be the one you come running to <end>',
 '<start> i ll never be untrue oh baby <end>',
 '<start> let s , let s stay together gether <end>',
 '<start> lovin you whether , whether <end>']

In [19]:
#tokenize() 함수로 데이터를 Tensor로 변환
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, #단어장의 크기:12,000 이상 
        filters=' ',
        oov_token="<unk>"
    )
    
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)  # corpus를 Tensor로 변환 
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2  64  16 ...   0   0   0]
 [  2 576   7 ...   0   0   0]
 [  2  26  25 ...   0   0   0]
 ...
 [  2  42 904 ...   0   0   0]
 [  2  42  67 ...   0   0   0]
 [  2   8  83 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f82aff9da90>


In [20]:
#단어사전 구축 인덱스 확인
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

#생성된 텐서를 소스와 타겟으로 분리해 모델 학습
src_input = tensor[:, :-1]   #소스 문장을 생성
tgt_input = tensor[:, 1:]   #타겟 문장을 생성

print(src_input[0])
print(tgt_input[0])

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to
[  2  64  16 215 277   5   4   5  23   5  23  31  14  37  30   7   3   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[ 64  16 215 277   5   4   5  23   5  23  31  14  37  30   7   3   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]


In [21]:
#데이터셋 객체 생성
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1   

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset element_spec=(TensorSpec(shape=(256, 32), dtype=tf.int32, name=None), TensorSpec(shape=(256, 32), dtype=tf.int32, name=None))>

In [22]:
#총 데이터의 20% 를 평가 데이터셋
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2)

In [27]:
enc_train.shape


(127100, 32)

In [28]:
dec_train.shape

(127100, 32)

In [23]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256  # 워드 벡터의 차원수, 즉 단어가 추상적으로 표현되는 크기
hidden_size = 1024  #모델에 얼마나 많은 일꾼을 둘 것인가
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [24]:
# 데이터셋에서 데이터 한 배치만 불러오기
for src_sample, tgt_sample in dataset.take(1): break

model(src_sample)

<tf.Tensor: shape=(256, 32, 12001), dtype=float32, numpy=
array([[[ 1.2905369e-04, -4.6051946e-04,  1.5081636e-04, ...,
          1.4101599e-04,  1.3058504e-05, -7.2385861e-05],
        [ 2.0605205e-04, -9.7505096e-04,  2.3316887e-04, ...,
          3.1202257e-04,  2.9182751e-04, -4.4447670e-04],
        [ 2.2592454e-04, -1.4872287e-03,  2.7402752e-04, ...,
          5.0595886e-04,  6.8606983e-04, -9.5890084e-04],
        ...,
        [-4.6140529e-04, -1.3787589e-04, -4.7785777e-04, ...,
         -4.6344236e-03,  5.3898239e-04,  3.1513225e-03],
        [-4.5685857e-04, -1.1829572e-04, -4.9138023e-04, ...,
         -4.6474207e-03,  5.1479711e-04,  3.2146764e-03],
        [-4.5256535e-04, -1.0250652e-04, -5.0339615e-04, ...,
         -4.6565263e-03,  4.9075380e-04,  3.2693848e-03]],

       [[ 1.2905369e-04, -4.6051946e-04,  1.5081636e-04, ...,
          1.4101599e-04,  1.3058504e-05, -7.2385861e-05],
        [ 2.1202574e-04, -7.6431117e-04,  9.9779943e-05, ...,
         -7.8295634e-05, 

In [25]:
model.summary()

Model: "text_generator_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     multiple                  3072256   
                                                                 
 lstm_2 (LSTM)               multiple                  5246976   
                                                                 
 lstm_3 (LSTM)               multiple                  8392704   
                                                                 
 dense_1 (Dense)             multiple                  12301025  
                                                                 
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [29]:
#모델 학습하기
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, 
          dec_train, 
          epochs=10,
          batch_size=256,
          validation_data=(enc_val, dec_val),
          verbose=1)

Epoch 1/10
497/497 [==============================] - 112s 221ms/step - loss: 1.2135 - val_loss: 1.2281
Epoch 2/10
497/497 [==============================] - 109s 220ms/step - loss: 1.1451 - val_loss: 1.1990
Epoch 3/10
497/497 [==============================] - 107s 216ms/step - loss: 1.0921 - val_loss: 1.1768
Epoch 4/10
497/497 [==============================] - 109s 220ms/step - loss: 1.0431 - val_loss: 1.1597
Epoch 5/10
497/497 [==============================] - 109s 220ms/step - loss: 0.9979 - val_loss: 1.1462
Epoch 6/10
497/497 [==============================] - 109s 219ms/step - loss: 0.9557 - val_loss: 1.1354
Epoch 7/10
497/497 [==============================] - 109s 220ms/step - loss: 0.9168 - val_loss: 1.1268
Epoch 8/10
497/497 [==============================] - 108s 217ms/step - loss: 0.8805 - val_loss: 1.1192
Epoch 9/10
497/497 [==============================] - 109s 219ms/step - loss: 0.8466 - val_loss: 1.1148
Epoch 10/10
497/497 [==============================] - 109s 220m

In [30]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    while True:
        # 입력받은 문장의 텐서를 입력
        predict = model(test_tensor) 
        # 예측된 값 중 가장 높은 확률인 word index를 뽑기
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 예측된 word index를 문장 뒤에 붙임
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마침
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [31]:
#문장 생성 함수 실행
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you <end> '

[참조](https://github.com/yenaryu/AI/blob/main/%5BE-04%5Dlyricist_ai.ipynb)